<a href="https://colab.research.google.com/github/bamtak/masakhane/blob/master/en_yo_jw300_notebook_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "yo" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [21]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-yo-baseline


In [22]:
#TODO: Skip for retrain
# Install opus-tools
! pip install opustools-pkg 

In [23]:
#TODO: Skip for retrain
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-yo.xml.gz not found. The following files are available for downloading:

        ./JW300_latest_xml_en.zip already exists
        ./JW300_latest_xml_yo.zip already exists
   4 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-yo.xml.gz

   4 MB Total size
./JW300_latest_xml_en-yo.xml.gz ... 100% of 4 MB
gzip: JW300_latest_xml_en-yo.xml already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [6]:
#TODO: Skip for retrain
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-04-12 10:38:37--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.02s   

2020-04-12 10:38:38 (17.4 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-04-12 10:38:40--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-yo.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [7]:
#TODO: Skip for retrain
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [25]:
#TODO: Skip for retrain
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 5663/474986 lines since contained in test set.


,source_sentence,target_sentence
0,Using Ladders — Do You Make These Safety Checks ?,Lílo Àkàbà — Ǹjẹ́ O Máa Ń Ṣe Àyẹ̀wò Wọ̀nyí Tó...
1,By Awake !,Látọwọ́ akọ̀ròyìn Jí !
2,correspondent in Ireland,ní Ireland


In [26]:
print(df.shape)

(469324, 2)


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [27]:
#TODO: Skip for retrain
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [28]:
#TODO: Skip for retrain
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 3.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144791 sha256=a1c171e3c3fe2182059c6a83ce2ae60e74751a04db7685b0ea524689d1f62695
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.14 0.00 percent complete
00:00:19.48 0.24 percent complete
00:00:38.24 0.48 percent complete
00:00:57.35 0.72 percent complete
00:01:16.41 0.96 percent complete
00:01:36.32 1.20 percent complete
00:01:54.42 1.43 percent complete
00:02:13.76 1.67 percent complete
00:02:32.86 1.91 percent complete
00:02:51.77 2.15 percent complete
00:03:10.64 2.39 percent complete
00:03:29.13 2.63 percent complete
00:03:48.32 2.87 percent complete
00:04:08.03 3.11 percent complete
00:04:27.79 3.35 percent complete
00:04:47.30 3.59 percent complete
00:05:06.58 3.82 percent complete
00:05:26.26 4.06 percent complete
00

00:08:01.48 5.98 percent complete
00:08:21.53 6.22 percent complete
00:08:40.78 6.45 percent complete
00:09:00.17 6.69 percent complete
00:09:19.45 6.93 percent complete
00:09:39.29 7.17 percent complete
00:09:59.40 7.41 percent complete
00:10:18.60 7.65 percent complete
00:10:38.11 7.89 percent complete
00:10:57.05 8.13 percent complete
00:11:16.07 8.37 percent complete
00:11:34.85 8.61 percent complete
00:11:54.55 8.84 percent complete
00:12:13.53 9.08 percent complete
00:12:33.17 9.32 percent complete
00:12:52.62 9.56 percent complete


00:13:11.97 9.80 percent complete
00:13:30.86 10.04 percent complete
00:13:50.00 10.28 percent complete
00:14:08.82 10.52 percent complete
00:14:27.63 10.76 percent complete
00:14:46.80 11.00 percent complete
00:15:06.59 11.24 percent complete
00:15:25.99 11.47 percent complete
00:15:45.38 11.71 percent complete
00:16:04.40 11.95 percent complete
00:16:24.45 12.19 percent complete
00:16:44.30 12.43 percent complete
00:17:03.79 12.67 percent complete
00:17:23.06 12.91 percent complete
00:17:42.35 13.15 percent complete
00:18:01.42 13.39 percent complete
00:18:20.67 13.63 percent complete
00:18:39.97 13.86 percent complete


00:18:59.55 14.10 percent complete
00:19:18.33 14.34 percent complete


00:19:37.46 14.58 percent complete
00:19:56.46 14.82 percent complete
00:20:15.45 15.06 percent complete
00:20:34.26 15.30 percent complete
00:20:53.51 15.54 percent complete
00:21:12.89 15.78 percent complete
00:21:33.00 16.02 percent complete
00:21:51.93 16.26 percent complete
00:22:11.19 16.49 percent complete
00:22:30.35 16.73 percent complete
00:22:49.70 16.97 percent complete
00:23:08.82 17.21 percent complete
00:23:27.89 17.45 percent complete
00:23:46.97 17.69 percent complete
00:24:06.84 17.93 percent complete
00:24:27.01 18.17 percent complete
00:24:46.83 18.41 percent complete
00:25:06.58 18.65 percent complete
00:25:25.83 18.88 percent complete
00:25:44.76 19.12 percent complete
00:26:04.07 19.36 percent complete
00:26:23.82 19.60 percent complete
00:26:43.21 19.84 percent complete
00:27:02.60 20.08 percent complete
00:27:21.80 20.32 percent complete
00:27:40.93 20.56 percent complete
00:28:00.87 20.80 percent complete


...

01:52:21.23 82.95 percent complete
01:52:40.37 83.19 percent complete
01:52:59.68 83.43 percent complete
01:53:19.09 83.67 percent complete
01:53:38.76 83.90 percent complete
01:53:58.42 84.14 percent complete


01:54:17.73 84.38 percent complete
01:54:37.15 84.62 percent complete
01:54:56.52 84.86 percent complete
01:55:15.57 85.10 percent complete
01:55:35.53 85.34 percent complete
01:55:54.96 85.58 percent complete
01:56:14.84 85.82 percent complete
01:56:34.90 86.06 percent complete
01:56:55.15 86.30 percent complete
01:57:14.53 86.53 percent complete
01:57:33.78 86.77 percent complete
01:57:53.07 87.01 percent complete
01:58:12.39 87.25 percent complete
01:58:31.64 87.49 percent complete
01:58:50.68 87.73 percent complete
01:59:09.88 87.97 percent complete
01:59:29.71 88.21 percent complete
01:59:48.88 88.45 percent complete
02:00:08.40 88.69 percent complete
02:00:27.13 88.92 percent complete


02:00:46.08 89.16 percent complete
02:01:05.66 89.40 percent complete
02:01:25.05 89.64 percent complete
02:01:44.92 89.88 percent complete
02:02:04.86 90.12 percent complete
02:02:24.68 90.36 percent complete
02:02:43.69 90.60 percent complete
02:03:03.36 90.84 percent complete
02:03:22.94 91.08 percent complete
02:03:42.05 91.32 percent complete
02:04:01.85 91.55 percent complete
02:04:20.88 91.79 percent complete
02:04:40.39 92.03 percent complete
02:04:59.71 92.27 percent complete
02:05:19.84 92.51 percent complete
02:05:39.14 92.75 percent complete
02:05:59.21 92.99 percent complete
02:06:18.53 93.23 percent complete
02:06:38.62 93.47 percent complete
02:06:59.31 93.71 percent complete
02:07:19.17 93.94 percent complete
02:07:38.72 94.18 percent complete
02:07:58.08 94.42 percent complete
02:08:17.85 94.66 percent complete
02:08:37.15 94.90 percent complete
02:08:57.10 95.14 percent complete
02:09:16.13 95.38 percent complete


02:09:36.09 95.62 percent complete
02:09:55.25 95.86 percent complete
02:10:14.43 96.10 percent complete
02:10:34.19 96.34 percent complete
02:10:53.38 96.57 percent complete
02:11:12.67 96.81 percent complete
02:11:32.19 97.05 percent complete
02:11:51.43 97.29 percent complete


02:12:11.22 97.53 percent complete
02:12:31.26 97.77 percent complete
02:12:50.80 98.01 percent complete
02:13:10.16 98.25 percent complete
02:13:29.41 98.49 percent complete
02:13:48.66 98.73 percent complete
02:14:08.34 98.96 percent complete
02:14:27.76 99.20 percent complete
02:14:48.28 99.44 percent complete
02:15:08.40 99.68 percent complete
02:15:28.13 99.92 percent complete


In [29]:
#TODO: Skip for retrain
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
TRAIN YOUR CHILDREN : “ I teach my children to check the expiration date of any packaged food items , such as snacks , before they buy them . ” ​ — Ruth , Nigeria
When she replied that she was , he explained that he and his mother were trying to assist his sister with a school report on Canadians .
Through the prophet Zephaniah , Jehovah answers : “ That day is a day of fury , a day of distress and of anguish , a day of storm and of desolation , a day of darkness and of gloominess , a day of clouds and of thick gloom . ”
We do not require that people simply do as we tell them , but we give them convincing reasons to obey Christ’s command .
Still , Jehovah can annihilate any rebel in the lake of fire , denying him any hope of a resurrection .
Lucaris was arrested , and on July 27 , 1638 , he was taken on board a small boat as if for banishment .
Yes , Jehovah remembered their faithful course .
□ To appear tough
During the ensuing confrontation , we Witnesses had to make



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [12]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2380 (delta 0), reused 0 (delta 0), pack-reused 2377
Receiving objects: 100% (2380/2380), 2.60 MiB | 4.24 MiB/s, done.
Resolving deltas: 100% (1670/1670), done.
Processing /content/joeynmt
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 276kB 24.4MB/s 
     |████████████████████████████████| 307kB 55.1MB/s 
     |████████████████████████████████| 17.1MB 201kB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 215kB 54.1MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 747kB 54.4MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73768 sha256=09642adb3c413596b594cde12da531b842bf2007c75f258d0eed07fa3b152a3d
  Stored in directory: /tmp/pip-ephem-wheel-cac

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [30]:
#TODO: Skip for retrain
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path/vocab.txt"  # Herman

bpe.codes.4000	dev.en	     test.bpe.yo     test.yo	   train.en
dev.bpe.en	dev.yo	     test.en	     train.bpe.en  train.yo
dev.bpe.yo	test.bpe.en  test.en-any.en  train.bpe.yo
bpe.codes.4000	dev.en	test.bpe.en  test.en-any.en  train.bpe.yo  vocab.txt
dev.bpe.en	dev.yo	test.bpe.yo  test.yo	     train.en
dev.bpe.yo	models	test.en      train.bpe.en    train.yo
BPE Xhosa Sentences
A@@ p@@ at@@ a ńlá ti ìgbàgbọ́ ( Wo ìpín@@ rọ̀ 12 - 14 )
À@@ ṣí@@ borí ìgb@@ àlà ( Wo ìpín@@ rọ̀ 15 - 18 )
Mo ti rí i pé àwọn èèyàn máa ń fẹ́ gb@@ ọ́@@ rọ̀ wa tí wọ́n bá rí i pé a lóye Bíbélì dáadáa , a sì fẹ́ ran àwọn lọ́wọ́ . ”
I@@ dà ẹ̀mí ( Wo ìpín@@ rọ̀ 19 - 20 )
Ó dájú pé lọ́@@ lá ìt@@ ì@@ lẹ́yìn Jèhófà , a máa dúró gb@@ ọ@@ in - in , Èṣù ò sì ní rí wa gbé ṣe .
Combined BPE Vocab
œ@@
Ísír@@
Isra@@
̃
×
ô
ʺ
bítì
Pété@@
Jóò@@


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if 'best' in filename and filename.endswith(".ckpt"):
        return filename
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [32]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
# !cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

Last checkpoint : best.ckpt


In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 2                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language, model_temp_dir=model_temp_dir, last_checkpoint=last_checkpoint)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [34]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-04-12 14:27:14,687 Hello! This is Joey-NMT.
2020-04-12 14:27:14.845045: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-04-12 14:27:16,195 Total params: 12188160
2020-04-12 14:27:16,197 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_

In [0]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"

In [36]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 385000	Loss: 41967.25391	PPL: 4.09053	bleu: 30.03572	LR: 0.00000415	
Steps: 386000	Loss: 42548.08594	PPL: 4.17107	bleu: 29.91872	LR: 0.00000415	
Steps: 387000	Loss: 42807.03906	PPL: 4.20748	bleu: 29.87333	LR: 0.00000415	
Steps: 388000	Loss: 42958.75781	PPL: 4.22896	bleu: 29.81204	LR: 0.00000415	
Steps: 389000	Loss: 43055.49609	PPL: 4.24271	bleu: 29.80630	LR: 0.00000415	
Steps: 390000	Loss: 43157.83984	PPL: 4.25731	bleu: 29.72661	LR: 0.00000291	
Steps: 391000	Loss: 43197.99609	PPL: 4.26306	bleu: 29.64012	LR: 0.00000291	
Steps: 392000	Loss: 43272.73438	PPL: 4.27376	bleu: 29.64561	LR: 0.00000291	
Steps: 393000	Loss: 43318.91797	PPL: 4.28040	bleu: 29.66845	LR: 0.00000291	
Steps: 394000	Loss: 43349.52344	PPL: 4.28479	bleu: 29.72774	LR: 0.00000291	


In [37]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-04-12 14:54:41,810 Hello! This is Joey-NMT.
2020-04-12 14:55:15,182  dev bleu:  31.03 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-04-12 14:55:53,503 test bleu:  38.62 [Beam search decoding with beam size = 5 and alpha = 1.0]
